<div align="center">

  <a href="https://github.com/leemjm92/STEAMxD" target="_blank">
    <img width="1024", src="https://user-images.githubusercontent.com/65292018/212471948-0bcf05da-5f10-45ae-a54a-38cf312c0909.jpg"></a>


<br>
  <a href="https://colab.research.google.com/github/leemjm92/STEAMxD/blob/main/training-custom-model.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
<br>

This <a href="https://github.com/ultralytics/yolov5">YOLOv5</a> 🚀 notebook by ROAR Lab presents simple train, validate and predict examples to help start your AI adventure.<br>Credit of this notebook goes to <a href="https://ultralytics.com/">Ultralytics</a> and <a href="https://roboflow.com/">Roboflow</a> as this notebook used both sample as reference.

</div>

# First time setup

Use this section to setup your GDrive for YOLOv5 usage with google colab for the first time. **Once this initial setup is done for further usage you will not be required to download the YOLOv5 repository again. Please use the next section [here](#future-setup) when accessing this colab notebook for future usage.**

In this section we're mounting GDrive, cloning GitHub [repository](https://github.com/ultralytics/yolov5), installing [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt), importing necessary libraries and checking PyTorch and GPU.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive
!mkdir -p steamxd
%cd steamxd
!mkdir -p datasets

In [ ]:
%cd steamxd/

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
from IPython.display import Image, clear_output  # to display images
display = utils.notebook_init()  # checks

# Future usage setup <a id="future-setup"></a>

If you already have YOLOv5 setup in your GDrive use this section to setup your colab notebook.

In this section we're mounting GDrive, installing [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt), importing necessary libraries and checking PyTorch and GPU.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
%cd /mydrive/steamxd/yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
from IPython.display import Image, clear_output  # to display images
display = utils.notebook_init()  # checks

# 1. Training custom model

Using the augmentation parameters that you've experimented in the [previous section](#image-aug), we will be training the custom model using your custom dataset.

<!---
To edit the instructions to include Augmentation discussions for ease of understanding


`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image 
                          vid.mp4  # video
                          path/  # directory
                          'path/*.jpg'  # glob
                          'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```
-->

## Extract training images

In [ ]:
%cd /mydrive/steamxd/datasets/baseline/
%ls

In [ ]:
# %cd "/mydrive/steamxd/datasets/sample/"
!unzip ./baseline-dataset.zip

In [ ]:
%ls

## Define the image parameters for training

### Define image augmentation parameters

In [ ]:
# how to rewrite the hyps yaml
%%writetemplate /mydrive/steamxd/yolov5/data/hyps/hyp.scratch-low.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for low-augmentation COCO training from scratch
# python train.py --batch 64 --cfg yolov5n6.yaml --weights '' --data coco.yaml --img 640 --epochs 300 --linear
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

#################################
### DO NOT ALTER THIS PORTION ###
#################################
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
#################################
### DO NOT ALTER THIS PORTION ###
#################################
###########################################
### Image Augmentation Parameters Start ###
###########################################
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 30.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.0  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.000  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.0  # image flip left-right (probability)
mosaic: 0.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
#########################################
### Image Augmentation Parameters End ###
#########################################
# copy_paste is for image segmentation leave this augmentation alone
copy_paste: 0.0  # segment copy-paste (probability)

### Define the image path and the number of classes

In [ ]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# COCO128 dataset https://www.kaggle.com/ultralytics/coco128 (first 128 images from COCO train2017) by Ultralytics
# Example usage: python train.py --data coco128.yaml
# parent
# ├── yolov5
# └── datasets
#     └── steamxd  ← downloads here (7 MB)
%%writetemplate /mydrive/steamxd/yolov5/data/baseline.yaml

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /mydrive/steamxd/datasets/baseline  # dataset root dir
train: train/images  # train images (relative to 'path') 128 images
val: train/images  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
names:
  0: humans
  1: pets

### Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

The only line of code that you've to edit is `nc:` this is the parameter for the number of classes (e.g. `nc: 3` means you have 3 different classes)

In [ ]:
##### this is for medium model #####
%%writetemplate /mydrive/steamxd/yolov5/models/baseline.yaml

# Parameters
nc: 2  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## How to start training on your custom *model*

In [ ]:
# ensure that you're in the parent yolov5 folder
%cd /mydrive/steamxd/yolov5

`--img`: to specific the input image size 640x640 \
`--batch`: the batch size of the image example input (16 images) [further explaination](https://developers.google.com/machine-learning/glossary#batch) \
`--epochs`: A full training pass over the entire training set such that each example has been processed once. [further explaination](https://developers.google.com/machine-learning/glossary#epoch) \
`--data`: configuration file for where the image dataset is stored \
`--cfg`: configuration file for where your model architecture \
`--weights`: pre-trained weights \
`--save-period`: by default the library only saves the best and last weights to save weights after every 5 epoch type `--save-period 5`\
`--cache`: store the training images in DRAM \
`--name`: this is the folder name for different training runs (e.g. aug-mosaic, aug-mixup, etc) \
`--project`: this is the directory path that you want to save your model runs in (e.g. '/mydrive/SUTD/STEAMxD') \
`--resume`: this will resume training of your model if training is interrupted (e.g. epochs 100, training stops at epochs 55.) If all epochs has been done (100/100) but you still wish to continue training on the existing model use the `--weights` flag instead (e.g. --weights /mydrive/SUTD/STEAMxD/yolov5/runs/train/exp) 

<!---
--project can be removed if I'm gonna just be cloning the yolov5 repo into gdrive
--resume flag to resume training if you're initial training epochs (300 and your model stops as 150)
--weights alternatively you can use the --weights flag to specific the weights file to continue training from
--data I need to add in the yaml file edit so that I can link it to the correct dataset
I should add in and image to explain the epochs and batch size
-->

In [ ]:
# give some explanation of the different parameters
!python train.py --img 640 --batch 16 --epochs 200 --data ./data/baseline.yaml --cfg ./models/baseline.yaml --weights yolov5m.pt --save-period 5 --cache --name baseline --project '/mydrive/steamxd/yolov5/runs/train'

### How to see metrics and evaluate your model

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
# if you see a google 403 error please ensure that your browser allows 3rd party cookie for tensorboard to be viewable
%load_ext tensorboard
%tensorboard --logdir /mydrive/steamxd/yolov5/runs/train/

In [ ]:
%cd /mydrive/steamxd/yolov5

In [ ]:
!python detect.py --weights './runs/train/baseline/weights/best.pt' --source '/mydrive/steamxd/datasets/sample/frame1.jpg'

In [ ]:
# print out test image
print("TEST IMAGE:")
Image(filename='/mydrive/steamxd/yolov5/runs/detect/exp/frame1.jpg', width=900)

### How to export your trained model weights

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')